# Find full-time software engineering jobs in US that require AWS skills

Learn how to use Exfunc and OpenAI to find full-time jobs on LinkedIn.

## Getting Started

### Install packages

In [ ]:
!pip install exfunc openai

### Configure API keys

You will need to provide API keys. You can get your Exfunc API key [here](https://exfunc.com) and your OpenAI API key [here](https://www.openai.com/).

Ensure both API keys are accessible in your local environment.

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key:\n")
if "EXFUNC_API_KEY" not in os.environ:
    os.environ["EXFUNC_API_KEY"] = getpass.getpass("Exfunc API key:\n")

### Configure clients

In [ ]:
from exfunc import Exfunc
from openai import OpenAI

exfunc = Exfunc()
openai = OpenAI()

## Finding full-time AWS jobs

### Search jobs

In [ ]:
response = exfunc.linkedin.search_job_postings(request={
    "keywords": "software engineer",
    "location": "united states",
    "job_type": "Full-time",
    "page": 1
})
job_posting_linkedin_urls = [job_posting.url for job_posting in response.job_postings]

### Get job details

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def get_job_posting(job_posting_url):
    try:
        response = exfunc.linkedin.get_job_posting(request={"job_posting_url": job_posting_url})
        return response.job_posting
    except:
        return None

job_postings = []
with ThreadPoolExecutor(max_workers=min(10, len(job_posting_linkedin_urls))) as executor:
    for job_posting in executor.map(get_job_posting, job_posting_linkedin_urls):
        if job_posting:
            job_postings.append(job_posting)

### Filter jobs that require AWS skills

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def qualify_job_posting(job_posting):
    prompt = f"Answer True or False: does the following job description match a full-time software engineering job that requires AWS skills?\n\nJob description:\n{job_posting.description}\n"
    response = openai.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gpt-4o-mini",
        temperature=0,
    )
    completion = response.choices[0].message.content.strip()
    return "true" in completion.lower()

qualified_job_postings = []
with ThreadPoolExecutor(max_workers=min(10, len(job_postings))) as executor:
    for i, is_qualified in enumerate(executor.map(qualify_job_posting, job_postings)):
        if is_qualified:
            qualified_job_postings.append(job_postings[i])

### View results

In [ ]:
import pandas as pd

records = [job_posting.model_dump() for job_posting in qualified_job_postings]

df = pd.DataFrame.from_records(records)
df

## Conclusion

You have now built a list of full-time software engineering jobs in US that require AWS skills.

If you have any questions or need help, feel free to reach out to us at support@exfunc.com